In [7]:
import pandas as pd

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,50,2024-02-06,-1,113.387755,98.306122,0.530367,10.830612,23.232653,0.20698,114.179592,...,11.307843,24.458824,0.182118,121.958824,0,2,1,2,11,2024
1,51,2024-02-06,-5,106.920000,98.286000,0.512800,12.462000,22.232000,0.18594,108.402000,...,11.004000,29.514000,0.208360,119.668000,0,2,1,2,15,2024
2,51,2024-02-06,3,111.560000,97.482000,0.531880,12.224000,25.868000,0.23194,113.750000,...,11.318000,22.358000,0.226300,114.462000,0,2,1,2,22,2024
3,51,2024-02-06,1,118.220000,99.550000,0.559360,10.706000,22.428000,0.20494,118.702000,...,10.553061,25.571429,0.181449,116.726531,0,2,1,1,7,2024
4,51,2024-02-06,1,112.920000,96.968000,0.564400,13.118000,22.804000,0.22212,116.092000,...,10.738000,24.252000,0.185940,114.852000,0,2,1,2,18,2024
5,51,2024-02-06,3,120.840000,98.776000,0.577060,10.604000,20.776000,0.20880,121.362000,...,13.001961,28.880392,0.213000,117.627451,0,2,1,2,21,2024
6,51,2024-02-06,-1,123.660000,101.214000,0.577100,11.002000,22.072000,0.23030,121.432000,...,12.656000,25.084000,0.239580,119.880000,0,2,1,2,17,2024


In [6]:
import joblib

In [8]:
def makeModelPredictions(games_df, odds_df, model_name='LR', show=True):
    if model_name == 'LR':
        model = joblib.load('Models/TrainedModels/LR_Model.pkl')
        scaler = joblib.load('Models/Scalers/LR.bin')
    elif model_name == 'RF':
        model = joblib.load('Models/TrainedModels/RF_Model.pkl')
        scaler = joblib.load('Models/Scalers/RF.bin')
    elif model_name == 'GB':
        model = joblib.load('Models/TrainedModels/GB_Model.pkl')
        scaler = joblib.load('Models/Scalers/GB.bin')
    elif model_name == 'SVM':
        model = joblib.load('Models/TrainedModels/SVM_Model.pkl')
        scaler = joblib.load('Models/Scalers/SVM.bin')
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    predictions = model.predict(scaled_df)
    
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [9]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-06,HOU,IND,3.750000,1.289855,0
1,2024-02-06,MEM,NYK,8.000000,1.090909,0
2,2024-02-06,ORL,MIA,2.300000,1.645161,0
3,2024-02-06,DAL,BRK,1.645161,2.300000,1
4,2024-02-06,MIN,CHI,1.487805,2.700000,0
5,2024-02-06,OKC,UTA,1.645161,2.300000,1
6,2024-02-06,MIL,PHO,2.400000,1.602410,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-06,HOU,IND,3.750000,1.289855,0
1,2024-02-06,MEM,NYK,8.000000,1.090909,0
2,2024-02-06,ORL,MIA,2.300000,1.645161,0
3,2024-02-06,DAL,BRK,1.645161,2.300000,0
4,2024-02-06,MIN,CHI,1.487805,2.700000,1
5,2024-02-06,OKC,UTA,1.645161,2.300000,0
6,2024-02-06,MIL,PHO,2.400000,1.602410,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-06,HOU,IND,3.750000,1.289855,0
1,2024-02-06,MEM,NYK,8.000000,1.090909,0
2,2024-02-06,ORL,MIA,2.300000,1.645161,0
3,2024-02-06,DAL,BRK,1.645161,2.300000,0
4,2024-02-06,MIN,CHI,1.487805,2.700000,1
5,2024-02-06,OKC,UTA,1.645161,2.300000,1
6,2024-02-06,MIL,PHO,2.400000,1.602410,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-06,HOU,IND,3.750000,1.289855,0
1,2024-02-06,MEM,NYK,8.000000,1.090909,0
2,2024-02-06,ORL,MIA,2.300000,1.645161,0
3,2024-02-06,DAL,BRK,1.645161,2.300000,0
4,2024-02-06,MIN,CHI,1.487805,2.700000,0
5,2024-02-06,OKC,UTA,1.645161,2.300000,1
6,2024-02-06,MIL,PHO,2.400000,1.602410,0
